# Setup a Pipeline and Stream CPU Utilization Data Using Model-driven Telemetry
##### <sup>Authored by Veena Manuel (veedas@cisco.com)</sup>

The use case illustrates how, with the YANG data model, you can stream telemetry data about CPU utilization. Monitoring CPU utilization ensures efficient storage capabilities in your network. This use case also illustrates setting up a pipeline to consume telemetry data that is streamed from the router.

The following table outlines the specifications of this use case:

| Mode            | Transport       | Encoding | 
|-----------------|---------------- | ---------|
| Dial-out        | gRPC            | GPB      |

> In a Dial-out mode, the router dials out to the receiver to establish a subscription-based telemetry session.

The following image represents a topology with 4 routers; router P1 is connected to a collector to stream telemetry data.
![Topology](./images/telemetry.png)

Telemetry involves the following workflow:
- Define: You define a subscription to stream data from the router to the receiver. To define a subscription,
you create a destination-group and a sensor-group.
- Deploy: The router establishes a subscription-based telemetry session and streams data to the receiver.
You verify subscription deployment on the router.
- Operate: You consume and analyse telemetry data using open-source tools, and take necessary actions
based on the analysis.

To gain an indepth understanding about telemetry streaming modes, see the [Telemetry Configuration Guide](https://www.cisco.com/c/en/us/support/routers/8000-series-routers/products-installation-and-configuration-guides-list.html).

## Configuration steps
* [Access Device Consoles](#Access-Device-Consoles)
* [Configure Base Network](#Configure-Base-Network)
* [Setup the Pipeline](#Setup-the-Pipeline)
* [Configure a Telemetry Session Between the Router and the Collector](#step3)
* [Verify the Telemetry configuration](#step4)
* [View the streamed telemetry data](#step5)
* [Operate on Telemetry Data](#step6)

### Access Device Consoles
> Play the following cell to access the SSH console of each device in the topology.

In [ ]:
from lib.xr import *
nodes = {
         'PE1':'', 
         'P1':'',
         'P2':'', 
         'PE2':'', 
         'trex':''
        }

tb = access_device_consoles("lib/tb.yaml", nodes)

### Configure Base Network
>A simple 4 router OSPF MPLS network is the base network. Play the below cell to configure the routers. We have defined the base configuration for each of the routers in the file [xr.py](./lib/xr.py) in the respective configuration strings.

In [ ]:
out = nodes['PE1'].configure(pe1_config_str)
out = nodes['P1'].configure(p1_config_str)
out = nodes['P2'].configure(p2_config_str)
out = nodes['PE2'].configure(pe2_config_str)

## <a name="step2"></a>Setup the Pipeline

Pipeline is a well-written Golang–based code that consumes IOS XR telemetry streams directly from routers or indirectly from a pub/sub bus (Kafka). In this example, you setup the pipeline to stream data directly from the router. Once collected, the Pipeline performs transformations of the data and forwards the result to the configured consumer.

Pipeline supports different input transport formats from routers:
- TCP
- gRPC
- UDP

Pipeline supports these encoding formats:
- (compact) GPB
- KV-GPB
- JSON

Pipeline supports streaming telemetry data to the following consumers:
- InfluxDB (TSDB)
- Prometheus (TSDB)
- Apache Kafka
- dump-to-file (mostly for diagnostics purposes)

> In this use case, the data will be dumped to a file.

### Create a file to stream telemetry data

Create a file dumpdata.txt to collect data that is streamed from the router. 

In [ ]:
import paramiko
from paramiko_expect import SSHClientInteraction

PROMPT = '.*root@.*'

collector_ipaddress = str(nodes['trex'].connections.cli.ip)
collector_port = str(nodes['trex'].connections.cli.port)

client = paramiko.SSHClient()
# Set SSH key parameters to auto accept unknown hosts
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=collector_ipaddress, port=collector_port, username='root', password='cisco123', allow_agent=False, look_for_keys=False)

# Check if the file is copied
interact = SSHClientInteraction(client, display=True)
interact.expect(PROMPT)

interact.send('cd /')
interact.expect(PROMPT)

# Create the file to dump data
interact.send('touch dumpdata.txt')
interact.expect(PROMPT)

# Create the file to dump data
interact.send('ls -l')
interact.expect(PROMPT)

### Copy the pipeline files to the server
> A pipeline is uploaded to the linux server when you play the below code-cell. Pipeline file used in the below code cell is from the github repository - https://github.com/CiscoDevNet/bigmuddy-network-telemetry-pipeline/tree/final

In [ ]:
# Using SFTP for the pipeline file-transfers. 
# Pipeline is a large file so this step takes approx 8-10 minutes to copy pipeline file to the server.

transfer = client.open_sftp()
transfer.put("./lib/pipeline_gpb_grpc.conf", "/pipeline_gpb_grpc.conf")
transfer.put("./lib/pipeline", "/pipeline")
transfer.close()

# Check if the file is copied
interact.send('chmod 777 pipeline')
interact.expect(PROMPT)

interact.send('ls -l pipeline*')
interact.expect(PROMPT)

interact.send('ifconfig')
interact.expect(PROMPT)

### Run the pipeline

In [ ]:
# Run the copied file.
interact.send('./pipeline -config pipeline_gpb_grpc.conf')
interact.expect('Wait for.*')

## <a name="step3"></a>Configure a telemetry session between the router and the collector using Yang Model

To configure telemetry on the router, you will need to setup the following values for a dial-out mode:
- Create a destination group: Create one or more destinations to collect telemetry data from a router. Define a destination-group to contain the details about the destinations. Include the destination address (ipv4 or ipv6), port, transport, and encoding format in the destination-group.

- Create a Sensor-group: Specify the subset of the data that you want to stream from the router using sensor paths. The Sensor path represents the path in the hierarchy of a YANG data model. Create a sensor-group to contain the sensor paths.
> Here, the sensor path is for streaming CPU utilization.

- Create a Subscription: Subscribe to telemetry data that is streamed from a router. A Subscription binds the destination-group with the sensor-group and sets the streaming method. The streaming method can be Cadence-driven or Event-driven telemetry. Cadence-driven telemetry continually streams data (operational statistics and state transitions) at a configured cadence. Whereas, event-driven telemetry optimizes data that is collected at the receiver and streams data only when a state transition occurs.

>Port number range <1-65535>.

> IMPORTANT: To find the IP address of the collector, check the output of the <b>ifconfig</b> command on the Linux server.

In [ ]:
from ncclient import manager
from ncclient.xml_ import to_ele
from lxml import etree

telemetry = '''
<config>
<telemetry-model-driven xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-telemetry-model-driven-cfg">
   <destination-groups>
    <destination-group>
     <destination-id>D1</destination-id>
     <ipv4-destinations>
      <ipv4-destination>
       <ipv4-address>192.168.122.104</ipv4-address>
       <destination-port>20030</destination-port>
       <encoding>gpb</encoding>
       <protocol>
        <protocol>grpc</protocol>
        <no-tls/>
       </protocol>
      </ipv4-destination>
     </ipv4-destinations>
    </destination-group>
   </destination-groups>
   <sensor-groups>
    <sensor-group>
     <sensor-group-identifier>SGroup1</sensor-group-identifier>
     <sensor-paths>
      <sensor-path>
       <telemetry-sensor-path>Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization</telemetry-sensor-path>
      </sensor-path>
     </sensor-paths>
    </sensor-group>
   </sensor-groups>
   <enable></enable>
   <subscriptions>
    <subscription>
     <subscription-identifier>Sub1</subscription-identifier>
     <sensor-profiles>
      <sensor-profile>
       <sensorgroupid>SGroup1</sensorgroupid>
       <sample-interval>3000</sample-interval>
      </sensor-profile>
     </sensor-profiles>
     <destination-profiles>
      <destination-profile>
       <destination-id>D1</destination-id>
      </destination-profile>
     </destination-profiles>
    </subscription>
   </subscriptions>
  </telemetry-model-driven>
 </config>
 '''
P1_ipaddress = str(nodes['P1'].connections.cli.ip)
P1_port = str(nodes['P1'].connections.cli.port)

def connect(host, port, user, password, source):
      conn = manager.connect(host=host,
                             port=port,
                             username=user,
                             password = password,
                             device_params={'name': 'iosxr'},
                             hostkey_verify=False,
                             allow_agent=False,
                             look_for_keys=False
                            )

      rpc_reply = conn.edit_config(config=telemetry)
      conn.commit()
      print(rpc_reply)

connect(P1_ipaddress, P1_port, 'cisco', 'cisco123', 'candidate')

Run the following command till the <i>Sensor Path State</i> is <b>Resolved</b>. Note that the state changes takes a few seconds. After the state changes to Resolved, proceed to the next step.

In [ ]:
# Show the telemetry configuration applied on the router.
out = nodes['P1'].execute('show tele mod sensor-group SGroup1 internal')

## <a name="step4"></a>Verify the telemetry configuration

Access the telnet consoles of the router to apply and verify the telemetry configuration.

In [ ]:
# Show the telemetry configuration applied on the router.
out = nodes['P1'].execute('show running-config telemetry model-driven')

Verify the deployment of the subscription and the telemetry configuration on the router.

In [ ]:
# Show the telemetry configuration applied on the router.
out = nodes['P1'].execute('show telemetry model-driven subscription Sub1')

## <a name="step5"></a>View the streamed telemetry data

After the telemetry session is established, the router streams data to the receiver to create a data lake. Check the collector console to see the streamed data. Here is the data from the Linx server (collector) console. The collector was configured to steam data to <b>dumpdata.txt file</b>. View its contents from the server console by playing the below cell. Click the stop button in the notebook when you want to stop the cell execution.

In [ ]:
out = nodes['trex'].execute('cat /dumpdata.txt')

> Snapshot of the data captured from the server.

![StreamedData](./images/dumpdata.png)

>Press <b>Ctrl + C</b> to stop the pipeline and the data stream if you are directly accessing the server console.

![StopPipeline](./images/stoppipeline.png)

## <a name="step6"></a>Operate on telemetry data

You can start consuming and analyzing telemetry data from the data lake using an open-sourced collection
stack based on your requirement. 
For example, the following tools can be used to analyze the collected telemetry data:
- Pipeline is a lightweight tool used to collect data. You can download Network Telemetry Pipeline from Github. You define how you want the collector to interact with routers and where you want to send the
processed data using pipeline.conf file.
- Telegraph (plugin-driven server agent) and InfluxDB (a time series database (TSDB)) stores telemetry
data, which is retrieved by visualization tools. You can download InfluxDB from Github. You define
what data you want to include into your TSDB using the metrics.json file.
- Grafana is a visualization tool that displays graphs and counters for data streamed from the router.

For more information, see http://xrdocs.io/telemetry/.

#### Clean-up the topology

>Once you are done with experimenting on the topology, you should bring down the emulator by executing the following steps:

In [ ]:
# Send ctrl-C when you want to stop the execution of pipeline.
interact.send("\x03")
interact.expect(PROMPT)

interact.send('exit')
interact.expect('.*')

client.close()


In [ ]:
out = nodes['PE1'].configure(unconfig_str)
out = nodes['P1'].configure(unconfig_str)
out = nodes['P2'].configure(unconfig_str)
out = nodes['PE2'].configure(unconfig_str)
out = nodes['P1'].configure('''
no telemetry model-driven
''')

Hope you now have a good idea on Model Driven Telemetry Configuration with Yang Models. If you have any comments or suggestions about this notebook, please reach out to mig-notebooks@cisco.com. We look forward to your feedback.